In [9]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
ss = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

# Add S3 paths 

In [10]:
mfcc = ss.read.csv('mfcc_final.csv', header=True)
cqt = ss.read.csv('/Users/RusL/Desktop/audio-project/cqt_final.csv', header=True)
gender = ss.read.csv('Lab41-SRI-VOiCES-speaker-gender-dataset.csv', header=True)

In [ ]:
cqt.cache()
gender.cache()
mfcc.cache()

# Use the cell below to drop useless columns

For example, there might be 'Unnamed: 0'

In [13]:
cqt.columns # to see what columns are there
cqt = cqt.drop(...)
# Repeat this with mfcc and gender

['Unnamed: 0',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',

# Process the filename and get the speaker id

In [4]:
# define a udf that extracts the speaker id out of the filename
def get_speaker(filename):
    i = filename.index('sp')
    return int(filename[i+2:i+6])
speaker = udf(get_speaker)

In [5]:
# process speaker id
cqt = cqt.withColumn('Speaker', speaker('FileName')).drop('FileName')
mfcc = mfcc.withColumn('Speaker', speaker('FileName')).drop('FileName')

# Join the tables and check the result

In [6]:
# join the dataframe
results = gender.join(mfcc, on='Speaker', how='rightOuter')\
                .join(cqt, on='Speaker', how='inner')

In [7]:
results.show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+--------------------+------+---------------+
|Speaker|          Centroid|          variance|          skewness|          kurtosis|              mfcc1|             mfcc2|              mfcc3|             mfcc4|              mfcc5|              mfcc6|              mfcc7|              mfcc8|              mfcc9|             mfcc10|             mfcc11|             mfcc12|      roll_off_max|      roll_off_min|            flatness|   zeroCrossingRate|                 rms|Gender|        DataSet|
+-------+------------------+------------------+------------------+------------------+-------------------+-

In [44]:
# check the number of observations
print(results.count())
print(cqt.count())

9381
9381


In [45]:
sc.stop()